# Package Imports

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn as sns

% matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

import sys
sys.path.append("..")

In [2]:
import thor

thor.setupOorb()
config = thor.Config()

In [3]:
DATABASE = "../data/msst_survey.db"
con = sql.connect(DATABASE)

## Config

In [4]:
from thor import Config

## Plotting Code

In [5]:
from thor.plotting import plotProjections
from thor.plotting import plotProjections3D
from thor.plotting import plotObservations
from thor.plotting import plotObservations3D
from thor.plotting import plotBinnedContour
from thor.plotting import plotScatterContour
from thor.plotting import plotCell

## Classes 

In [6]:
from thor import Cell
from thor import TestParticle

## Functions

In [7]:
from thor import findAverageObject
from thor import findExposureTimes
from thor import buildCellForVisit
from thor import rangeAndShift
from thor import clusterAndLink
from thor import analyzeClusters
from thor import runRangeAndShiftOnVisit

# Load Data

In [8]:
observationsNoNoise = pd.read_sql("""SELECT * FROM observations""", con)
noise = pd.read_sql("""SELECT * FROM noise_100""", con)
noise["obsId"] = np.arange(observationsNoNoise["obsId"].values[-1] + 1, observationsNoNoise["obsId"].values[-1] + 1 + len(noise))

In [9]:
observations = pd.concat([observationsNoNoise, noise], sort=False)
observations.reset_index(inplace=True, drop=True)
del observationsNoNoise
del noise

In [10]:
survey = pd.read_sql("""SELECT * FROM survey""", con)

In [11]:
orbits = pd.read_sql("""SELECT * FROM mpcOrbitCat""", con)
# Only grab the orbits of objects with observations
orbits = orbits[orbits["designation"].isin(observations["designation"].unique())]

In [12]:
neos = orbits[orbits["a_au"] <= 1.3]["designation"].values

In [13]:
findAverageObject(observations[observations["visitId"] == 1])

THOR: findAverageObject
-------------------------
p3143 is the most average object.
-------------------------



'p3143'

In [14]:
projected_obs = runRangeAndShiftOnVisit(observations, 1, 0, 0, useAverageObject=True, searchArea=10, cellArea=10)

THOR: runRangeAndShiftOnVisit
-------------------------
Running Thor on visit 1...
Search cell area: 10 
Search cell shape: square 
Cell area: 10 
Cell shape: square 

THOR: findAverageObject
-------------------------
p3143 is the most average object.
-------------------------

THOR: rangeAndShift
-------------------------
Running range and shift...
Assuming r = 2.7617290181 AU
Assuming v = [-0.00964426 -0.00379003  0.00017245] AU per day
Preparing rotation matrices...
Convering to ecliptic coordinates...
Calculating object to observer unit vector...
Calculating object to observer distance assuming r = 2.7617290181 AU...
Calculating object to observer position vector...
Calculating barycentic object position vector...
Calculating vector normal to plane of orbit...
Calculating R1 rotation matrix...
Calculating R2 rotation matrix...
Calculating final rotation matrix...
Done.

THOR: findExposureTimes
-------------------------
Generating particle ephemeris for the middle of every night.
Fi

In [18]:
allClusters, clusterMembers = clusterAndLink(
    projected_obs, 
    eps=0.005, 
    minSamples=5, 
    vxRange=[-0.1, 0.1], 
    vyRange=[-0.1, 0.1], 
    vxBins=100, 
    vyBins=100, 
    saveFiles=["../analysis/msst/visit1_p3143/allClusters.txt", 
               "../analysis/msst/visit1_p3143/clusterMembers.txt" ],
    #vxValues=vxValues,
    #vyValues=vyValues,
    threads=1)

THOR: clusterAndLink
-------------------------
Running velocity space clustering...
X velocity range: [-0.1, 0.1]
X velocity bins: 100
Y velocity range: [-0.1, 0.1]
Y velocity bins: 100
User defined x velocity values: False
User defined y velocity values: False
Velocity grid size: 10000
Max sample distance: 0.005
Minimum samples: 5
Done. Completed in 1646.9682939052582 seconds.

Restructuring clusters...
Done. Completed in 0.3041880130767822 seconds.

Found 8128 clusters.
Total time in seconds: 1647.272882938385
Saving allClusters to ../analysis/msst/visit1_p3143/allClusters.txt
Saving clusterMembers to ../analysis/msst/visit1_p3143/clusterMembers.txt
-------------------------



In [20]:
allClusters, clusterMember, allObjects, summary = analyzeClusters(
    projected_obs, 
    allClusters, 
    clusterMembers, 
    partialThreshold=1.0, 
    #saveFiles=["../analysis/msst/visit1_U1469/allClustersAnalysis.txt", 
    #           "../analysis/msst/visit1_U1469/clusterMembersAnalysis.txt",
    #           "../analysis/msst/visit1_U1469/allObjects.txt",
    #           "../analysis/msst/visit1_U1469/summary.txt"],
    minSamples=5)

THOR: analyzeClusters
-------------------------
Analyzing observations...
Object observations: 21840
Noise observations: 11822
Observation contamination (%): 35.11971956508823
Unique objects: 3055
Unique objects with at least 5 detections: 1856
Unique objects with at least 100.0% of 5 detections: 1856

Analyzing clusters...
Pure clusters: 1480
Partial clusters: 0
Duplicate visit clusters: 3806
False clusters: 6648
Total clusters: 8128
Cluster contamination (%): 81.79133858267717
Unique linked objects: 834
Unique missed objects: 1022
Completeness (%): 44.935344827586206
Done.
Total time in seconds: 0.26240110397338867
-------------------------



In [ ]:
ids = clusterMembers[clusterMembers["cluster_id"].isin(allClusters[allClusters["pure"] == 1]["cluster_id"].values)]["obs_id"].values

In [ ]:
plotProjections(projected_obs1[projected_obs1["obsId"].isin(ids)], colorByObject=True)

In [ ]:
plotProjections3D(projected_obs1[~projected_obs1["obsId"].isin(ids)], colorByObject=True)

In [ ]:
from thor.plotting import _setAxes

def plotCluster(clusterId, allClusters, clusterMembers, observations, columnMapping=Config.columnMapping):
    
    
    # Grab the observation IDs in the cluster and the velocity
    obs_ids = clusterMembers[clusterMembers["cluster_id"] == clusterId]["obs_id"].values
    vx = allClusters[allClusters["cluster_id"] == clusterId]["theta_vx"].values
    vy = allClusters[allClusters["cluster_id"] == clusterId]["theta_vx"].values
    
    # Grab the initial exposure time
    mjd0 = observations[columnMapping["exp_mjd"]].min()
    
    # Grab the actual observations and their projection space position
    cluster_obs = observations[observations["obsId"].isin(obs_ids)]
    obs_ids = cluster_obs[columnMapping["obs_id"]].values
    theta_x = cluster_obs["theta_x_deg"].values
    theta_y = cluster_obs["theta_y_deg"].values
    mjd = cluster_obs[columnMapping["exp_mjd"]].values
    
    # Grab non cluster observations for plotting purposes
    non_cluster_obs = observations[~observations["obsId"].isin(obs_ids)]
    
    dt = mjd - mjd0
    xx = theta_x - vx * dt
    yy = theta_y - vy * dt
    
    fig, ax = plt.subplots(1, 2, dpi=200)
    ax[0].scatter(*non_cluster_obs[["theta_x_deg", "theta_y_deg"]].values.T, c="b", s=0.05)
    ax[0].scatter(*cluster_obs[["theta_x_deg", "theta_y_deg"]].values.T, c="r", s=0.05)
    ax[0].set_aspect("equal")
    _setAxes(ax[0], "gnomonic")
    
    ax[1].scatter(xx, yy, s=1)
    ax[1].set_aspect("equal")
    ax[1].set_xlabel(r"$\theta_X$  - $v_x t$ [deg]")
    ax[1].set_ylabel(r"$\theta_Y$  - $v_y t$ [deg]")
    
    fig.suptitle("Cluster ID: {}\n".format(clusterId))
    
    #plotProjections(cluster_obs, colorByObject=True)
    #plotProjections3D(cluster_obs, colorByObject=True)


In [ ]:
plotCluster(2, allClusters, clusterMembers, projected_obs)

In [ ]:
fig = plotProjections(
    projected_obs[projected_obs["designation"].isin(allObjects[allObjects["findable"] == 1]["designation"])],
    colorByObject=True)

In [ ]:
fig = plotProjections(
    projected_obs[projected_obs["designation"].isin(allObjects[allObjects["found"] == 1]["designation"])],
    colorByObject=True)

In [ ]:
projected_obs[projected_obs["name"].isin()]

In [ ]:
found = orbits[orbits["designation"].isin(allObjects[allObjects["found"] == 1]["designation"])]
missed = orbits[orbits["designation"].isin((allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["designation"]))]

In [ ]:
fig, ax = plotScatterContour(found, 
                             "a_au",
                             "i_deg",
                             "e",
                             plotCounts=False, 
                             logCounts=True, 
                             countLevels=4, 
                             mask=None,
                             xLabel="a [AU]",
                             yLabel="i [deg]",
                             zLabel="e",
                             scatterKwargs={"s": 1, "vmin": 0, "vmax": 1})
                            
o = orbits[orbits["designation"] == avg_obj]
ax.text(ax.get_xlim()[-1] - 0.40 * (ax.get_xlim()[1] - ax.get_xlim()[0]), ax.get_ylim()[1] - 0.05 * ax.get_ylim()[1], "Found objects: {}".format(len(found)))
ax.scatter(o["a_au"].values, o["i_deg"].values, c="r", s=20, marker="+")
ax.set_title("Recovered Orbits\nVisit: {}, Object: {}".format(1, avg_obj))
#fig.savefig("../analysis/msst/plots/manual_recovered_visit1_u1469_2.png")

In [ ]:
observations

In [ ]:
fig, ax = plotScatterContour(missed, 
                             "a_au",
                             "i_deg",
                             "e",
                             plotCounts=False, 
                             logCounts=True, 
                             countLevels=4, 
                             mask=None,
                             xLabel="a [AU]",
                             yLabel="i [deg]",
                             zLabel="e",
                             scatterKwargs={"s": 1, "vmin": 0, "vmax": 1})
o = orbits[orbits["designation"] == avg_obj]
ax.text(ax.get_xlim()[-1] - 0.40 * (ax.get_xlim()[1] - ax.get_xlim()[0]), ax.get_ylim()[1] - 0.05 * ax.get_ylim()[1], "Missed objects: {}".format(len(missed)))
ax.scatter(o["a_au"].values, o["i_deg"].values, c="r", s=20, marker="+")
ax.set_title("Missed Orbits\nVisit: {}, Object: {}".format(1, avg_obj))
#fig.savefig("../analysis/msst/plots/manual_missed_visit1_u1469_2.png")

In [ ]:
avg_obj = "U1469"

In [ ]:
found_obs = projected_obs1[projected_obs1["designation"].isin(allObjects[allObjects["found"] == 1]["designation"])]
missed_obs = projected_obs1[projected_obs1["designation"].isin((allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["designation"]))]

In [ ]:
fig, ax = plotScatterContour(found_obs, 
                             *Config.v[0:3],  
                             countLevels=4, 
                             xLabel="dx/dt [AU per day]",
                             yLabel="dy/dt [AU per day]",
                             zLabel="dz/dt [AU per day]")
obs = projected_obs[projected_obs["designation"] == avg_obj]
ax.text(ax.get_xlim()[-1] - 0.40 * (ax.get_xlim()[1] - ax.get_xlim()[0]), ax.get_ylim()[1] - 0.05 * ax.get_ylim()[1], "Missed objects: {}".format(len(missed)))
ax.scatter(*obs[Config.v[0:2]].values.T, c="r", s=1, marker="+")
ax.set_title("Found Orbits\nVisit: {}, Object: {}".format(1, avg_obj))
#fig.savefig("../analysis/msst/plots/manual_missed_visit1_u1469_2.png")

In [ ]:
fig, ax = plotScatterContour(missed_obs, 
                             *Config.v[0:3],  
                             countLevels=4, 
                             #mask=(objects["r_au"] < 5),
                             xLabel="dx/dt [AU per day]",
                             yLabel="dy/dt [AU per day]a",
                             zLabel="dz/dt [AU per day]")
ax.scatter(*obs[Config.v[0:2]].values.T, c="r", s=1, marker="+")
ax.text(ax.get_xlim()[-1] - 0.40 * (ax.get_xlim()[1] - ax.get_xlim()[0]), ax.get_ylim()[1] - 0.30 * ax.get_ylim()[1], "Missed objects: {}".format(len(missed)))
ax.set_title("Missed Orbits\nVisit: {}, Object: {}".format(1, avg_obj))
#fig.savefig("../analysis/msst/plots/manual_missed_visit1_u1469_2.png")

In [ ]:
fig, ax = plotBinnedContour(found_obs, 
                             *Config.v[0:3],  
                             countLevels=4, 
                             xLabel="dx/dt [AU per day]",
                             yLabel="dy/dt [AU per day]",
                             zLabel="dz/dt [AU per day]")
obs = projected_obs[projected_obs["designation"] == avg_obj]
ax.text(ax.get_xlim()[-1] - 0.40 * (ax.get_xlim()[1] - ax.get_xlim()[0]), ax.get_ylim()[1] - 0.05 * ax.get_ylim()[1], "Missed objects: {}".format(len(missed)))
ax.scatter(*obs[Config.v[0:2]].values.T, c="r", s=1, marker="+")
ax.set_title("Missed Orbits\nVisit: {}, Object: {}".format(1, avg_obj))
fig.savefig("../analysis/msst/plots/manual_missed_visit1_u1469.png_2")

In [ ]:
fig, ax = plt.subplots(2, 2, dpi=200)
fig.set_size_inches(10, 10)
ax[0, 0].scatter(missed["a_au"].values, 
                 missed["i_deg"].values, 
                c="k", 
                alpha=0.5,
                s=1)
ax[0, 0].scatter(found["a_au"].values, 
                found["i_deg"].values, 
                c="r", 
                alpha=0.5,
                s=1)
ax[0, 0].set_xlabel("a [AU]")
ax[0, 0].set_ylabel("i [deg]")
ax[0, 0].hlines(o["i_deg"].values, ax[0, 0].get_xlim()[0], ax[0, 0].get_xlim()[1], lw=1, color="b")
ax[0, 0].vlines(o["a_au"].values, ax[0, 0].get_ylim()[0], ax[0, 0].get_ylim()[1], lw=1, color="b")

ax[0, 1].scatter(missed["e"].values, 
                 missed["i_deg"].values, 
                c="k", 
                alpha=0.5,
                s=1)
ax[0, 1].scatter(found["e"].values, 
                found["i_deg"].values, 
                c="r", 
                alpha=0.5,
                s=1)
ax[0, 1].set_xlabel("e")
ax[0, 1].set_ylabel("i [deg]")
ax[0, 1].hlines(o["i_deg"].values, ax[0, 1].get_xlim()[0], ax[0, 1].get_xlim()[1], lw=1, color="b")
ax[0, 1].vlines(o["e"].values, ax[0, 1].get_ylim()[0], ax[0, 1].get_ylim()[1], lw=1, color="b")

ax[1, 0].scatter(missed["a_au"].values, 
                missed["e"].values, 
                c="k", 
                alpha=0.5,
                s=1)
ax[1, 0].scatter(found["a_au"].values, 
                 found["e"].values, 
                 c="r", 
                 alpha=0.5,
                 s=1)
ax[1, 0].set_xlabel("a [AU]")
ax[1, 0].set_ylabel("e")
ax[1, 0].hlines(o["e"].values, ax[1, 0].get_xlim()[0], ax[1, 0].get_xlim()[1], lw=1, color="b")
ax[1, 0].vlines(o["a_au"].values, ax[1, 0].get_ylim()[0], ax[1, 0].get_ylim()[1], lw=1, color="b")
fig.suptitle("Missed versus Recovered")



#cb = fig.colorbar(cm)

In [ ]:
projected_obs["visitId"]["designation"].value_counts()

In [ ]:
fig, ax = plt.subplots(2, 2, dpi=200)
fig.set_size_inches(10, 10)
ax[0, 0].scatter(missed["a_au"].values, 
                 missed["i_deg"].values, 
                c="k", 
                alpha=0.5,
                s=1)
ax[0, 0].scatter(found["a_au"].values, 
                found["i_deg"].values, 
                c="r", 
                alpha=0.5,
                s=1)
ax[0, 0].set_xlabel("a [AU]")
ax[0, 0].set_ylabel("i [deg]")



ax[0, 1].scatter(missed["e"].values, 
                 missed["i_deg"].values, 
                c="k", 
                alpha=0.5,
                s=1)
ax[0, 1].scatter(found["e"].values, 
                found["i_deg"].values, 
                c="r", 
                alpha=0.5,
                s=1)
ax[0, 1].set_xlabel("e")
ax[0, 1].set_ylabel("i [deg]")

ax[1, 0].scatter(missed["a_au"].values, 
                missed["e"].values, 
                c="k", 
                alpha=0.5,
                s=1)
ax[1, 0].scatter(found["a_au"].values, 
                 found["e"].values, 
                 c="r", 
                 alpha=0.5,
                 s=1)
ax[1, 0].set_xlabel("a [AU]")
ax[1, 0].set_ylabel("e")

ax[1, 1].scatter(missed["a_au"].values, 
                 missed["e"].values, 
                 c="k", 
                 alpha=0.5,
                s=1)
ax[1, 1].scatter(found["a_au"].values, 
                 found["e"].values, 
                 c="r", 
                 alpha=0.5,
                 s=1)


#cb = fig.colorbar(cm)

In [ ]:
pd.DataFrame(columns=["test"])